# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Chroma

In [ ]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=my_emb
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
question = "VMAX上网日志业务是什么？"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

打印一下检索到的内容

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

/tmp/ipykernel_429407/3999574677.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
/tmp/ipykernel_429407/3999574677.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [2]:
results = vectordb.similarity_search(query="VMAX", k=2)
# results

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [3]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1)

# my_llm.invoke("你好")

/tmp/ipykernel_429407/1202256217.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1)


## 3. 构建检索问答链

prompts

In [4]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [5]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [6]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"query": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['result']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

/tmp/ipykernel_429407/2682531927.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})  # Pass string directly, not dict


回答：<think>
好的，我现在要回答用户的问题：“什么是VMAX？”根据提供的上下文，我需要先理解VMAX是什么以及它在什么背景下使用。

首先，从文档中多次出现“ZXVMAX-S”这个名称，看起来像是一个系统或平台的名称。文档提到了多个功能模块，如采集层、存储共享层和应用分析层，这些都表明这是一个复杂的软件系统，可能用于数据分析和网络性能监控。

接下来，文档详细描述了VMAX的功能，包括语音业务分析、一键投诉处理、区域感知功能、用户感知评估等。此外，还提到了支持多种数据存储方式，如MPP数据库Gbase和HDFS，并且有客户端和REST接口供用户访问。

综合这些信息，可以推测VMAX是一个用于移动网络性能监控和优化的系统，帮助运营商分析网络问题，提升用户体验。它可能整合了来自不同网元的数据，进行处理、存储和分析，并提供直观的仪表盘和报告。

因此，总结起来，VMAX很可能是一个专业的网络数据分析平台，专为电信行业设计，用于实时监控、故障诊断和优化建议。
</think>

**VMAX** 是一个专业的网络数据分析与性能管理平台，主要用于移动通信网络的监控、分析和优化。它整合了来自不同网元的数据，通过采集层、存储共享层和应用分析层进行处理，并提供用户友好的界面和API接口供运营商使用。其功能包括一键投诉处理、区域感知评估、用户质量分析等，帮助提升网络性能和用户体验。


回答：<think>
好的，我现在需要回答用户的问题：“有哪些功能”。根据提供的上下文，我应该找出ZXVMAX-S系统中提到的所有功能。

首先，我看到文档中有多个章节和子节，每个部分都提到了不同的功能。例如，在4.1节中提到了用户分析功能、业务分析功能、网络分析功能和专题分析功能。另外，还有可维可测功能和支持保障功能等。

接下来，我要仔细阅读每个部分，确保不遗漏任何功能点。比如，一键投诉功能是在4.1.1提到的，区域感知功能在4.1.2，用户感知功能在4.1.3，业务感知功能在4.1.4，可维可测功能在4.1.5。

此外，还有一些其他的功能，如拨测结果自动比对、NAT日志入库、北向接口等。这些可能是在不同的子节中提到的，需要一一列出。

最后，我应该将所有找到的功能整理成一个清晰的列表，并按照出现的顺序排列，这样用户看起来会更直观。
</think>

ZXVMAX-S系统提供了多种

In [7]:
question_1 = "什么是vmax的上网日志系统？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
<think>
好的，我现在要回答用户的问题：“什么是VMAX的上网日志系统？”首先，我需要理解这个问题。看起来用户对VMAX这个系统的功能和用途不太清楚，特别是关于上网日志的部分。

从提供的资料来看，VMAX全称为ZXVMAX-S多维价值分析系统，主要用于处理和存储用户的上网日志数据。它能够采集、存储、分析这些日志，并支持多种查询方式，帮助运营商追溯用户行为。

接下来，我需要整理出关键点：功能模块、数据采集方式、存储方式、查询方法以及与其他系统的对接。这些都是解释VMAX上网日志系统的重要部分。

首先，功能模块方面，系统分为采集层、存储共享层和应用分析层。每个层次都有不同的组件，比如探针用于采集信令，Saturn负责处理数据存入数据库等。这些细节可以帮助用户理解系统的结构和运作方式。

然后是数据采集，VMAX支持硬采和软采两种方式。硬采通过分光或镜像设备采集网络流量，而软采则是与网元直接接口获取信令数据。这两种方法各有优缺点，可能适用于不同的网络环境，这也是一个重要的信息点。

存储方面，系统使用Gbase数据库和HDFS分布式文件系统，确保了数据的长期保存和高效管理。同时，支持MPP数据库，这在处理大量数据时非常有用。

查询功能也是关键部分，VMAX提供了多种基于时间、IP、IMSI等条件的查询方式，满足不同需求。此外，北向接口的支持使其能够与运营商的查询平台集成，进一步扩展了系统的应用范围。

最后，系统还支持与其他省级网关对接，并提供拨测结果比对功能，这在质量监控和问题排查中非常有用。

总结来说，VMAX是一个全面的上网日志管理系统，具备强大的数据处理能力和灵活的应用接口，能够满足运营商在用户行为分析、网络优化等方面的需求。通过详细解释这些方面，可以清晰地回答用户的问题。
</think>

ZXVMAX-S多维价值分析系统（简称“VMAX”）是一款用于管理和分析用户上网日志的系统。它主要用于电信运营商，帮助他们记录和查询用户的上网行为数据。

### 主要功能：
1. **数据采集**：
   - 通过硬采或软采方式采集网络信令。
   - 硬采：使用分光或镜像设备采集Gn口、S1-U等接口的信令。
   - 软采：与GGSN/xGW等网元直接对接获取信令数据。

2. **日志存储**：


In [9]:
question_2 = "严威是谁？"
result = qa_chain({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
<think>

</think>

对不起，我还没有学会回答这个问题。如果你有其他问题，我非常乐意为你提供帮助。


### 4.2 无知识库大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [6]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_23612\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
# 修改后的Prompt模板（添加chat_history变量）
template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
# 创建对话链
qa_chain = ConversationalRetrievalChain.from_llm(
        llm=my_llm,
        retriever=vectordb.as_retriever(),
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_PROMPT},
        chain_type="stuff"
    )
    
    # result = qa_chain({"question": question})
    # return result["answer"]
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
questions = [
    "什么是VMAX？",
    "有哪些功能", 
    # "整理成表格"  
]

for question in questions:
    result = qa_chain({"question": question})  # Pass string directly, not dict
    # print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

回答：VMAX是ValueMAX端到端多维价值分析系统的简称，面向用户的网络运维和运营分析产品。它立足于从客户角度去感知和分析网络和业务信息，并通过实时的分析、事后分析等方式为运营商提供全面支撑。


